The code for this recommender system was adapted from: https://www.youtube.com/watch?v=V2h3IOBDvrA&feature=youtu.be&list=PLfYUBJiXbdtS2UQRzyrxmyVHoGW0gmLSM&t=6147

In [72]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import ( Input, Embedding, Flatten, Dot, Add, Concatenate, 
        Dense, Dropout, Activation, Multiply )
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tf_print import tf_print

In [2]:
ratings = pd.read_csv('rratings.csv', sep=',',encoding='latin-1')
ratings.head()

,userId,routeId,rating
0,cjtdagjg2dgc90b52si9pjzhm,cjtdahb82hbx50b57evpzr1aq,3.9
1,cjtdagjg2dgc90b52si9pjzhm,cjtdahb82hbx40b57idzxb9cp,2.5
2,cjtdagjg2dgc90b52si9pjzhm,cjtdahb82hbww0b57nh810qhn,2.3
3,cjtdagjg2dgc90b52si9pjzhm,cjtdahb82hby30b57udf930sv,1.7
4,cjtdagjg2dgc90b52si9pjzhm,cjtdahb82hby00b570fr577pz,1.1


In [3]:
u_unique = ratings.userId.unique()
user2Id = { userId : index for index, userId in enumerate(u_unique)}
ratings.userId = ratings.userId.apply(lambda x: user2Id[x])

r_unique = ratings.routeId.unique()
route2Id = { routeId : index for index, routeId in enumerate(r_unique)}
ratings.routeId = ratings.routeId.apply(lambda x: route2Id[x])

In [8]:
n_users = ratings.userId.nunique()
n_routes = ratings.routeId.nunique()
n_users, n_routes

(30, 60)

In [59]:
n_factors = 50

In [60]:
np.random.seed = 42

In [61]:
msk = np.random.rand(len(ratings)) < 0.8

In [62]:
trn = ratings[msk]
val = ratings[~msk]
val.shape

(95, 3)

In [63]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

# Neural Network Model

In [67]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-5)
route_in, r = embedding_input('route_in', n_routes, n_factors, 1e-5)

In [68]:
x = Concatenate()([u, r])
x = Flatten()(x)
x = Dense(units=(n_factors * 2), activation='relu')(x)
x = Dropout(rate=0.75)(x)
x = Dense(1)(x)
model = tf.keras.Model(inputs=[user_in, route_in], outputs=x)
model.compile(optimizer=Adam(lr=0.05), loss='mse')

In [69]:
model.fit([trn.userId, trn.routeId], trn.rating, batch_size=64, epochs=20,
          validation_data=([val.userId, val.routeId], val.rating))

Train on 346 samples, validate on 95 samples
Epoch 1/20
346/346 [==============================] - 2s 5ms/sample - loss: 6.7072 - val_loss: 2.4678
Epoch 2/20
346/346 [==============================] - 0s 57us/sample - loss: 3.3685 - val_loss: 1.1625
Epoch 3/20
346/346 [==============================] - 0s 112us/sample - loss: 2.8901 - val_loss: 1.5058
Epoch 4/20
346/346 [==============================] - 0s 90us/sample - loss: 2.7860 - val_loss: 1.5148
Epoch 5/20
346/346 [==============================] - 0s 113us/sample - loss: 2.5269 - val_loss: 1.0130
Epoch 6/20
346/346 [==============================] - 0s 100us/sample - loss: 2.4151 - val_loss: 1.6070
Epoch 7/20
346/346 [==============================] - 0s 103us/sample - loss: 2.4158 - val_loss: 1.1982
Epoch 8/20
346/346 [==============================] - 0s 133us/sample - loss: 2.2216 - val_loss: 1.4063
Epoch 9/20
346/346 [==============================] - 0s 125us/sample - loss: 1.9582 - val_loss: 1.0420
Epoch 10/20
346/346 [==

In [71]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_in (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
route_in (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_24 (Embedding)        (None, 1, 50)        1500        user_in[0][0]                    
__________________________________________________________________________________________________
embedding_25 (Embedding)        (None, 1, 50)        3000        route_in[0][0]                   
__________________________________________________________________________________________________
concatenat

In [73]:
tf_print(tf.math.sqrt(0.9505))

0.9749359
